**Pandas Profiling - Censo 2014, 2017 e 2021**

In [ ]:
!pip install --upgrade ydata-profiling

In [ ]:
# ==============================================================================
# 0. SETUP - Importação das Bibliotecas
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Configurações visuais para os gráficos
sns.set_theme(style="whitegrid", palette="plasma")
plt.rcParams['figure.figsize'] = [14, 8]

In [ ]:
# ==============================================================================
# 1. CONFIGURAÇÕES - Caminhos para os arquivos
# ==============================================================================
PATHS = {
    2021: {
        'cursos': '/content/sample_data/censo/2021/MICRODADOS_CADASTRO_CURSOS_2021.CSV',
        'ies': '/content/sample_data/censo/2021/MICRODADOS_CADASTRO_IES_2021.CSV'
    },
    2017: {
        'cursos': '/content/sample_data/censo/2017/MICRODADOS_CADASTRO_CURSOS_2017.CSV',
        'ies': '/content/sample_data/censo/2017/MICRODADOS_CADASTRO_IES_2017.CSV'
    },
    2014: {
        'cursos': '/content/sample_data/censo/2014/MICRODADOS_CADASTRO_CURSOS_2014.CSV',
        'ies': '/content/sample_data/censo/2014/MICRODADOS_CADASTRO_IES_2014.CSV'
    }
}

ESTADOS_NORDESTE = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']
CURSOS_COMPUTACAO_PATH = '/content/sample_data/censo/cursos.csv'




In [ ]:
# ==============================================================================
# 2. CARGA E JUNÇÃO
# ==============================================================================

# Caminho para o arquivo com os cursos válidos
CURSOS_COMPUTACAO_PATH = '/content/sample_data/censo/cursos.csv'  # atualize conforme necessário

# Carrega os cursos válidos a partir do CSV
df_cursos_validos = pd.read_csv(CURSOS_COMPUTACAO_PATH, encoding='latin1')
df_cursos_validos.columns = df_cursos_validos.columns.str.upper().str.strip()
cursos_validos = df_cursos_validos['ID;NO_CURSO'].str.split(';').str[1].str.upper().unique().tolist()

lista_df_anos = []
print("Iniciando carga e junção dos dados...")

for ano, paths in PATHS.items():
    try:
        print(f"\nProcessando {ano}...")
        df_cursos = pd.read_csv(paths['cursos'], sep=';', encoding='latin1', low_memory=False)
        df_ies = pd.read_csv(paths['ies'], sep=';', encoding='latin1', low_memory=False)

        df_cursos.columns = df_cursos.columns.str.upper()
        df_ies.columns = df_ies.columns.str.upper()

        df_ano = pd.merge(df_cursos, df_ies, on='CO_IES', how='left', suffixes=('_CURSO', '_IES'))
        df_ano['ANO'] = ano

        # Filtra apenas cursos de computação
        df_ano = df_ano[df_ano['NO_CURSO'].str.upper().isin(cursos_validos)]

        lista_df_anos.append(df_ano)
        print(f"  > {ano} OK.")
    except Exception as e:
        print(f"  > Erro em {ano}: {e}")

if not lista_df_anos:
    raise Exception("Nenhum dado carregado.")

df_censo_total = pd.concat(lista_df_anos, ignore_index=True)

# Filtrar apenas registros do Nordeste
df_ne = df_censo_total[df_censo_total['SG_UF_IES'].isin(ESTADOS_NORDESTE)].copy()

print(f"\nTotal de registros filtrados para o Nordeste (cursos de computação): {len(df_ne)}")

In [ ]:
# ==============================================================================
# 3. ANÁLISE DE DUPLICATAS
# ==============================================================================
print("\nVerificando duplicatas...")

duplicatas = df_ne.duplicated(subset=['ANO', 'CO_IES', 'CO_CURSO'], keep=False)
df_dups = df_ne[duplicatas]

print(f"Total de registros duplicados: {duplicatas.sum()}")

if not df_dups.empty:
    # Por curso
    print("\nDuplicatas por curso:")
    print(df_dups['NO_CURSO'].value_counts())

    # Por UF
    print("\nDuplicatas por UF:")
    print(df_dups['SG_UF_IES'].value_counts())

In [ ]:
# ==============================================================================
# 4. ANÁLISE EXPLORATÓRIA COMPARATIVA
# ==============================================================================

# Gráfico 1 - Cursos por Ano
plt.figure(figsize=(12, 7))
sns.countplot(x='ANO', data=df_ne, palette='magma')
plt.title('Quantidade de Cursos por Ano - Nordeste')
plt.xlabel('Ano')
plt.ylabel('Número de Cursos')
plt.show()


# Gráfico 2 - Categoria Administrativa
plt.figure(figsize=(14, 8))
#print(df_ne.columns[df_ne.columns.str.contains("CATEGORIA|IES", case=False)])
sns.countplot(y='TP_CATEGORIA_ADMINISTRATIVA_IES', hue='ANO', data=df_ne, palette='viridis')
plt.title('Cursos por Categoria da IES e Ano - Nordeste')
plt.xlabel('Qtd. Cursos')
plt.ylabel('Categoria')
plt.legend(title='Ano')
plt.show()

# Gráfico 3 - Modalidade de Ensino
if 'TP_MODALIDADE_ENSINO' in df_ne.columns:
    df_ne['MODALIDADE'] = df_ne['TP_MODALIDADE_ENSINO'].map({1: 'Presencial', 2: 'A Distância'})
    plt.figure(figsize=(12, 7))
    sns.countplot(x='ANO', hue='MODALIDADE', data=df_ne, palette='coolwarm')
    plt.title('Modalidade de Ensino por Ano - Nordeste')
    plt.xlabel('Ano')
    plt.ylabel('Qtd. Cursos')
    plt.legend(title='Modalidade')
    plt.show()
else:
    print("Coluna 'TP_MODALIDADE_ENSINO' ausente. Pulando gráfico de modalidade.")

# Gráfico 4 - Matrículas
if 'QT_MAT' in df_ne.columns:
    df_matriculas = df_ne.groupby('ANO')['QT_MAT'].sum().reset_index()
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_matriculas, x='ANO', y='QT_MAT', palette='flare')
    plt.title('Total de Matrículas por Ano - Nordeste')
    plt.xlabel('Ano')
    plt.ylabel('Matrículas')
    plt.show()
else:
    print("Coluna 'QT_MAT' ausente. Pulando gráfico de matrículas.")

# Gráfico 5 - Concluintes
if 'QT_CONC' in df_ne.columns:
    # Total por ano
    df_concluintes = df_ne.groupby('ANO')['QT_CONC'].sum().reset_index()
    sns.lineplot(data=df_concluintes, x='ANO', y='QT_CONC', marker='o', linewidth=3)
    plt.title('Total de Concluintes por Ano - Nordeste')
    plt.xlabel('Ano')
    plt.ylabel('Concluintes')
    plt.show()

    # Total por curso
    concluintes_curso = df_ne.groupby('NO_CURSO')['QT_CONC'].sum().sort_values(ascending=False)
    concluintes_curso.plot(kind='barh', color='skyblue')
    plt.title("Total de Concluintes por Curso (Nordeste)")
    plt.xlabel("Total de Concluintes")
    plt.ylabel("Curso")
    plt.show()

    # Média por ano, estado e curso
    media_concluintes = df_ne.groupby(['ANO', 'SG_UF_IES', 'NO_CURSO'])['QT_CONC'].mean().reset_index()
    plt.figure(figsize=(16, 8))
    sns.barplot(data=media_concluintes, x='ANO', y='QT_CONC', hue='NO_CURSO')
    plt.title("Média de Concluintes por Ano e Curso")
    plt.ylabel("Média de Concluintes")
    plt.xlabel("Ano")
    plt.legend(title="Curso")
    plt.show()

    # Desvio padrão por ano e curso
    desvio_concluintes = df_ne.groupby(['ANO', 'NO_CURSO'])['QT_CONC'].std().reset_index()
    print("\nDesvio padrão de concluintes por ano e curso:")
    print(desvio_concluintes)
else:
    print("Coluna 'QT_CONC' ausente. Pulando gráficos de concluintes.")

# Gráfico 6 - Cursos por Estado e Ano
plt.figure(figsize=(14, 8))
sns.countplot(data=df_ne, y='SG_UF_IES', hue='ANO', palette='cubehelix')
plt.title('Distribuição de Cursos por Estado e Ano (Nordeste)')
plt.xlabel('Qtd. Cursos')
plt.ylabel('Estado')
plt.legend(title='Ano')
plt.show()

In [ ]:
# ==============================================================================
# 5. RELATÓRIO COM YDATA PROFILING
# ==============================================================================
print("\nGerando relatório HTML do ydata_profiling...")

# Seleciona apenas as colunas de interesse
colunas_interesse = [
    'ANO', 'NO_CURSO', 'SG_UF_IES', 'TP_CATEGORIA_ADMINISTRATIVA_IES',
    'TP_MODALIDADE_ENSINO', 'QT_MAT', 'QT_CONC'
]
df_reduzido = df_ne[colunas_interesse].copy()

# Gera o relatório com base no DataFrame reduzido
profile = ProfileReport(
    df_reduzido,
    title="Análise Exploratória - Censo Educação Superior Nordeste (2014, 2017, 2021)",
    explorative=True,
    minimal=True
)
profile.to_file("relatorio_computacao_nordeste.html")
print("✅ Relatório salvo como 'relatorio_computacao_nordeste.html'.")
